In [1]:
# import topopy
from test_functions import *
# import matplotlib.pyplot as plt
# import samplers
# import seaborn as sns
# from matplotlib import patches, collections
import numpy as np
from ipywidgets import interact, HTML
import random
import pdir

In [2]:
from sidepanel import SidePanel

import ipywidgets as widgets

from regulus.utils import io
from regulus.topo import * 
from regulus.alg import *
from regulus.measures import *
from regulus.models import *
from regulus.tree import *

from ipyregulus import TreeWidget, TreeView, DetailsView, DataWidget
from ipyregulus.filters.filters import * 
from ipyregulus.filters.monitor import Monitor
from ipyregulus.tree.has_tree import *
from ipyregulus.alg.view import *

In [3]:
test_function = gerber_different_heights
samples = 200
X = np.random.rand(samples,2)
Y = test_function(X)

# Y = (Y - np.min(Y)) / (np.max(Y) - np.min(Y)) # Not sure whether this is needed 

all_pts = np.hstack((X,np.expand_dims(Y, axis=1)))

# X = samplers.CVTSampler.generate_samples(samples, 2, 1)


# msc = topopy.MorseSmaleComplex(graph="beta skeleton", gradient="steepest", max_neighbors=8, beta=1.0)
# msc.build(X, Y)


# NEW PART IN REG to create pts at runtime 

In [4]:
from regulus.topo import morse_smale, Regulus
from regulus.data.data import Data
from regulus.measures import *
from regulus.models import *
import pandas as pd

cols = ['x'+str(i) for i in range(all_pts.shape[1])]

cols[-1] = 'y'

df = pd.DataFrame(all_pts, columns=cols)

pts = Data.from_pts(df)

# Normalize here  
pts.normalize()

regulus = morse_smale(pts, knn=8)#**kwargs)

regulus.add_attr('linear', linear_model)
regulus.add_attr('fitness', fitness)
regulus.add_attr('relative_fitness', relative_fitness)

regulus.tree.add_attr('parent_fitness', parent_fitness)
regulus.tree.add_attr('child_fitness', child_fitness)
regulus.tree.add_attr('size', node_size)
regulus.tree.add_attr('rel_size', node_relative_size)
regulus.tree.add_attr('span', node_span)




found 8 singles
len(idx)= 198


/media/orpheus92/OS/conda/envs/regulus/lib/python3.6/site-packages/numpy/core/fromnumeric.py:51: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  return getattr(obj, method)(*args, **kwds)
/media/orpheus92/OS/conda/envs/regulus/lib/python3.6/site-packages/numpy/core/fromnumeric.py:51: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


In [5]:
from ipyregulus import TreeWidget, TreeView, DetailsView, DataWidget

tree = TreeWidget(regulus.tree)
data = DataWidget(data=regulus)


# TreeView

In [6]:
v2 = show_tree(tree)

w = widgets.Dropdown(
    options=['span', 'fitness', 'parent_fitness', 'child_fitness'],
    value='span',
    description='Attribute:',
    disabled=False,
)
panel = SidePanel(title='Fitness')#, side='split-bottom', ref='SidePanel')
with panel:
    display(w, v2.view, v2.filter)
widgets.link((w, 'value'), (v2.view, 'field'))

TreeView(field='span', tree_model=TreeWidget(attrs={'span': {0: 0.6797178418716125, 1: 0.05489350981834029, 2:…

# Details View

In [7]:
details = DetailsView(data=data)

p7 = SidePanel(title='details')
with p7:
    display(details)

# Linking Two Views

In [8]:

widgets.link((v2.view, 'details'), (details, 'show'))


# Create resample class, resampling a user specified partition

In [9]:
from resample import get_pts, Resample

# Change the following line to use differennt partition to resample
ind = get_pts(data.partitions[0],data.pts_loc)

selectP = df.loc[ind,:]
r = Resample(selectP,pred_method="bounding_box")

# A line chart for a specified partition, different interactions could be added 

In [10]:
from cyplot import plot
myp = plot(selectP,index = 'y',debug=True, dims=['x0','x1','y'])#ptype='PCA')#, ptype='line'/ PCA. line. dims=[]
myp.enable(['brush_x','brush'])
myp.on('brush_x', r.cb)
myp.on('brush', r.cb)

display(myp)

In [11]:
# from cyplot import plot
myp2 = plot(selectP,index = 'y',debug=True, ptype='PCA')#, ptype='line'/ PCA. line. dims=[]
myp2.enable(['brush_x','brush'])
myp2.on('brush_x', r.cb)
myp2.on('brush', r.cb)
display(myp2)

In [ ]:
myp2.enable('x_brush')
display(myp2)

In [ ]:
pdir(regulus.tree.attr.values)
regulus.tree.attr.values()['parent_fitness']

# random generate pts based on user selected range of function value

In [17]:
r.add_samples(5) # user's specified # of samples
r.x



[[0.16336204231323384, 0.055537546377243126],
 [0.5424718088861109, 0.10052110368288422],
 [0.3258533909078509, 0.38889215251199505],
 [0.6815924270323849, 0.6013073008119757],
 [0.6699362402803362, 0.2247699932307624]]

In [34]:
myp.fig.traits

<bound method HasTraits.traits of Figure(axes=[Axis(grid_lines='none', label='x0', scale=LinearScale(allow_padding=False), side='bottom'), Axis(grid_lines='none', label='x1', orientation='vertical', scale=LinearScale(allow_padding=False), side='left'), ColorAxis(orientation='vertical', scale=ColorScale(scheme='YlOrRd'), side='right')], fig_margin={'top': 0, 'bottom': 40, 'left': 50, 'right': 50}, interaction=BrushSelector(color='red', marks=[Scatter(color=array([0.57072211, 0.85112575, 0.4248163 , 0.74874592, 1.05133737,
       1.31793813, 0.91380612, 1.25084817, 1.39489838, 0.45243176,
       0.91257904, 1.36025941, 0.59876907, 0.98410549, 1.00349326,
       0.42114979, 0.9282147 , 0.56330083, 1.41439761, 0.89164359,
       0.67147186, 0.61432548, 0.81084219, 0.58790321, 0.99754027,
       0.59827649, 0.53362929, 1.37918171, 1.12118799, 0.81556179,
       0.97125254, 0.42692826, 1.09513888, 0.48463088, 1.03410736,
       0.52054281, 1.09943024, 0.93430875, 1.13164339, 1.10184812,
    

In [25]:
myp.old_enabled

[]

# When user adds enough samples, they could use regression to predict the point locations to resample 

In [19]:
# Would scale back if needed 
X2 = r.get_input()

new_x = np.vstack((X,X2))
new_y = test_function(new_x)

In [21]:
new_y.shape

(205,)

In [ ]:
all_pts

In [ ]:

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="dark")
rs = np.random.RandomState(50)

# Set up the matplotlib figure
f, axes = plt.subplots(3, 3, figsize=(9, 9), sharex=True, sharey=True)

# Rotate the starting point around the cubehelix hue circle
for ax, s in zip(axes.flat, np.linspace(0, 3, 10)):

    # Create a cubehelix colormap to use with kdeplot
    cmap = sns.cubehelix_palette(start=s, light=1, as_cmap=True)

    # Generate and plot a random bivariate dataset
    x, y = rs.randn(2, 50)
    sns.kdeplot(x, y, cmap=cmap, shade=True, cut=5, ax=ax)
    ax.set(xlim=(-3, 3), ylim=(-3, 3))

f.tight_layout()

# Bivariate Distribution

In [ ]:
mean, cov = [0, 1], [(1, .5), (.5, 1)]
data = np.random.multivariate_normal(mean, cov, 200)
df = pd.DataFrame(data, columns=["x", "y"])
sns.jointplot(x="x", y="y", data=df, kind="kde");
sns.kdeplot(df.x, df.y, ax=ax)


In [ ]:
import os
import ipywidgets as widgets

import bqplot.pyplot as plt
from bqplot import *

In [ ]:
image_path = os.path.abspath('KDE.png')

from bqplot import Image

with open(image_path, 'rb') as f:
    raw_image = f.read()
ipyimage = widgets.Image(value=raw_image, format='png')
scales = {'x': LinearScale(), 'y': LinearScale()}
image = Image(image=ipyimage, scales=scales)
fig = Figure(marks=[image], padding_x=0, padding_y=0, animation_duration=1000)
fig.axes = [Axis(scale=scales['x']), Axis(scale=scales['y'], orientation='vertical')]
fig


In [ ]:
from bqplot import DateScale, LinearScale, OrdinalScale, Axis, Lines, Scatter, Bars, Hist, Figure
from bqplot.interacts import (
    FastIntervalSelector, IndexSelector, BrushIntervalSelector,
    BrushSelector, MultiSelector, LassoSelector, PanZoom, HandDraw
)
from traitlets import link

from ipywidgets import ToggleButtons, VBox, HTML

#multi_sel = MultiSelector(scale=dt_x, marks=[lc2, lc3])
#br_intsel = BrushIntervalSelector(scale=lc1_x, marks=[lc2, lc3])
#index_sel = IndexSelector(scale=dt_x, marks=[lc2, lc3])
int_sel = FastIntervalSelector(scale=LinearScale(), marks=fig.marks, color='FireBrick')


from collections import OrderedDict
selection_interacts = ToggleButtons(options=OrderedDict([#('HandDraw1', hd), ('HandDraw2', hd2), ('PanZoom', pz), 
                                                       ('FastIntervalSelector', int_sel),# ('IndexSelector', index_sel),
                                                       #('BrushIntervalSelector', br_intsel), ('MultiSelector', multi_sel),
                                                       ('None', None)]))

deb = HTML()


link((selection_interacts, 'value'), (fig, 'interaction'))
VBox([fig, selection_interacts, deb], align_self='stretch')

In [ ]:
from ipywidgets import ToggleButtons, VBox, HTML, Layout

#fig = plt.figure(padding_x=0, padding_y=0)
#image = plt.imshow(ipyimage, 'widget')
b = HTML()

A = VBox([fig,b])

In [ ]:
fig.marks

In [ ]:
pdir(fig.interaction)

In [ ]:
pdir(myp.fig.interaction)

In [ ]:
myp.enabled

In [ ]:
selection_interacts.value

In [ ]:

from string import ascii_letters
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="white")

# Generate a large random dataset
rs = np.random.RandomState(33)
d = pd.DataFrame(data=rs.normal(size=(100, 26)),
                 columns=list(ascii_letters[26:]))

# Compute the correlation matrix
corr = d.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})